In [1]:
import cv2
import numpy as np
import pywt
from scipy.linalg import svd, diagsvd
from scipy.optimize import differential_evolution
from target import target_func

Optimal scaling factor: [0.1]
Function value: 0.0


In [2]:
def rgb_to_yiq(rgb_image):
    transformation_matrix = np.array([[0.299, 0.587, 0.114],
                                      [0.596, -0.275, -0.321],
                                      [0.212, -0.523, 0.311]])
    yiq_image = np.dot(rgb_image, transformation_matrix.T)
    return yiq_image

def yiq_to_rgb(yiq_image):
    transformation_matrix = np.array([[1.000, 0.956, 0.620],
                                      [1.000, -0.272, -0.647],
                                      [1.000, -1.108, 1.703]])
    rgb_image = np.dot(yiq_image, np.linalg.inv(transformation_matrix.T))
    return rgb_image

def apply_dwt(image):
    coeffs = pywt.dwt2(image, 'haar')
    cA, (cH, cV, cD) = coeffs
    return coeffs

def apply_svd(matrix):
    U, S, Vt = svd(matrix, full_matrices=False)
    return U, S, Vt

def watermark_embedding(U, S, Vt, watermark_matrix, scale_factor):
    print(f'S: {S}')
    print(f'SF: {scale_factor}')
    print(f'WM: ${watermark_matrix}')
    S_w = (scale_factor * watermark_matrix) + S
    watermarked_matrix = np.dot(U, np.dot(diagsvd(S_w, *U.shape), Vt))
    return watermarked_matrix

def differential_evolution_optimization(bounds):
    result = differential_evolution(target_func, bounds)
    return result.x

In [3]:
# 3 level DWT function

def haar_filter():
    # Haar wavelet filters
    lpf = np.array([1, 1]) / np.sqrt(2)
    hpf = np.array([1, -1]) / np.sqrt(2)
    return lpf, hpf

import numpy as np

def dwt_1d(signal, lpf, hpf):
    # Pad the signal if its length is not a power of 2
    if len(signal) % 2 != 0:
        signal = np.pad(signal, (0, 1))

    # Convolve with filters
    cA = np.convolve(signal, lpf, mode='valid')[::2]
    cD = np.convolve(signal, hpf, mode='valid')[::2]
    
    return cA, cD

def dwt_2d(signal, lpf, hpf):
    rows, cols = signal.shape
    
    # Apply 1D DWT along rows
    approximations_row = []
    details_row = []
    for i in range(rows):
        cA, cD = dwt_1d(signal[i], lpf, hpf)
        approximations_row.append(cA)
        details_row.append(cD)
    
    # Convert lists to arrays
    approximations_row = np.array(approximations_row)
    details_row = np.array(details_row)
    
    # Apply 1D DWT along columns
    approximations_col = []
    details_col = []
    
    rows, cols = approximations_row.shape
    for j in range(cols):
        cA, cD = dwt_1d(approximations_row[:, j], lpf, hpf)
        approximations_col.append(cA)
        details_col.append(cD)
    
    # Convert lists to arrays
    approximations_col = np.array(approximations_col)
    details_col = np.array(details_col)
    
    return approximations_col, details_col

def dwt(signal, levels=3):
    lpf, hpf = haar_filter()  # Assuming you have defined the Haar wavelet filters
    
    # Apply 2D DWT for the specified number of levels
    coeffs = []
    approximations = signal.copy()
    for _ in range(levels):
        cA, cD = dwt_2d(approximations, lpf, hpf)
        coeffs.append(cD)
        approximations = cA
    
    coeffs.append(approximations)  # Append the approximation coefficients at the last level
    return coeffs

In [4]:
host_image = cv2.imread('./assets/host_image.png')
watermark_image = cv2.imread('./assets/watermark_image.png', cv2.IMREAD_GRAYSCALE)

In [5]:
# Convert to YIQ color space

host_image_yiq = rgb_to_yiq(host_image / 255.0)  # Normalized

# Extract Y component (luminance)
host_image_yiq_y_comp = host_image_yiq[:, :, 0]
host_image_yiq_y_comp

# cv2.imshow("Host Image RGB", host_image)

# cv2.imshow("Host Image YIQ", host_image_yiq)

# cv2.waitKey(5000)

array([[0.54901961, 0.54901961, 0.54901961, ..., 0.35294118, 0.35294118,
        0.35294118],
       [0.54901961, 0.54901961, 0.54901961, ..., 0.35294118, 0.35294118,
        0.35294118],
       [0.52941176, 0.52941176, 0.56078431, ..., 0.35294118, 0.35294118,
        0.35294118],
       ...,
       [0.42745098, 0.42745098, 0.44313725, ..., 0.09411765, 0.09411765,
        0.09411765],
       [0.41568627, 0.41568627, 0.43137255, ..., 0.09019608, 0.09019608,
        0.09019608],
       [0.41568627, 0.41568627, 0.43137255, ..., 0.09019608, 0.09019608,
        0.09019608]])

In [6]:
# Apply 3-level DWT to the Y component

print(f'Y comp Host Image Shape :  {host_image_yiq_y_comp.shape}\n')

dwt_coeffs_y = dwt(host_image_yiq_y_comp, levels=3)

for i, coeffs in enumerate(dwt_coeffs_y):
    print(f"Level {i + 1} detail coefficients shape:", coeffs.shape)
    
dwt_coeffs_y

Y comp Host Image Shape :  (512, 512)

Level 1 detail coefficients shape: (256, 256)
Level 2 detail coefficients shape: (128, 128)
Level 3 detail coefficients shape: (64, 64)
Level 4 detail coefficients shape: (64, 64)


[array([[ 0.        ,  0.        ,  0.00784314, ...,  0.00784314,
          0.        ,  0.        ],
        [ 0.        ,  0.00392157,  0.00196078, ...,  0.00196078,
         -0.00392157,  0.        ],
        [ 0.        ,  0.00196078,  0.00196078, ...,  0.00392157,
         -0.02156863,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.00196078, ...,  0.00588235,
         -0.00392157,  0.        ],
        [ 0.        ,  0.01372549, -0.00588235, ...,  0.00196078,
         -0.00196078,  0.        ],
        [ 0.        ,  0.00392157,  0.00392157, ...,  0.        ,
         -0.00392157,  0.        ]]),
 array([[ 0.02941176, -0.00784314, -0.06372549, ...,  0.01862745,
         -0.0127451 , -0.00098039],
        [ 0.01862745,  0.00588235, -0.03333333, ...,  0.00588235,
         -0.03235294, -0.01960784],
        [ 0.00980392,  0.        , -0.01666667, ...,  0.04117647,
         -0.06960784, -0.12352941],
        ...,
        [ 0.08333333,  0.07843137,  0.00784314, ..., -

In [7]:
def show_dwt_coeffs(dwt_coeffs):
    # Normalize the coefficients to [0, 255]
    normalized_coeffs = []
    for coeff in dwt_coeffs:
        normalized_coeff = np.abs(coeff) / np.max(np.abs(coeff)) * 255
        normalized_coeffs.append(normalized_coeff.astype(np.uint8))
        
    # Show the normalized coefficients
    for i in range(len(normalized_coeffs)):
        cv2.imshow(f"Level {i + 1} coefficients", normalized_coeffs[i])
        cv2.waitKey(5000)
        cv2.destroyAllWindows()

    cv2.destroyAllWindows()
    
# show_dwt_coeffs(dwt_coeffs_y)

In [8]:
# Apply SVD

def svd_block(coeff_block):
    coeff_block_2d = np.atleast_2d(coeff_block)
    U, S, Vt = np.linalg.svd(coeff_block_2d, full_matrices=False)
    return U, S, Vt

def svd_dwt_coeffs(dwt_coeffs):
    svd_results = []
    for level_coeffs in dwt_coeffs:
        svd_level = []
        for coeff_block in level_coeffs:
            U, S, Vt = svd_block(coeff_block)
            svd_level.append((U, S, Vt))
        svd_results.append(svd_level)
    return svd_results

svd_results = svd_dwt_coeffs(dwt_coeffs_y)
svd_results

[[(array([[-1.]]),
   array([0.17436654]),
   array([[ 0.        ,  0.        , -0.04498075,  0.06747112,  0.        ,
            0.02249037,  0.        ,  0.06747112,  0.        ,  0.        ,
            0.02249037,  0.        ,  0.        , -0.15743261, -0.02249037,
           -0.02249037,  0.04498075,  0.04498075,  0.        , -0.04498075,
            0.04498075,  0.02249037,  0.04498075, -0.02249037, -0.02249037,
           -0.04498075,  0.02249037, -0.02249037,  0.08996149, -0.11245187,
            0.        ,  0.        ,  0.13494224, -0.02249037,  0.02249037,
            0.        , -0.02249037, -0.04498075,  0.        ,  0.11245187,
           -0.06747112,  0.        ,  0.06747112, -0.02249037,  0.08996149,
           -0.06747112,  0.04498075, -0.06747112, -0.02249037,  0.11245187,
           -0.06747112, -0.04498075,  0.02249037,  0.04498075, -0.04498075,
            0.        ,  0.04498075,  0.04498075, -0.04498075,  0.02249037,
           -0.11245187, -0.04498075, -0.04498

In [9]:
# Example scale factor (use DE to find optimal)

scale_factor = 0.05  # This should be optimized

In [10]:
# Watermark embedding

# watermarked_cA = watermark_embedding(U, S, Vt, watermark_image, scale_factor)